In [4]:
pip install hmmlearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.6/164.6 kB 3.5 MB/s eta 0:00:00


In [11]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

class IrisDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.FloatTensor(features).unsqueeze(1)
        self.labels = torch.LongTensor(labels)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_layers=1, dropout=0.1):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size * 2, num_classes)  # *2 for bidirectional

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

class EarlyStopper:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs, early_stopper):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    results = {
        'train_loss': [],
        'val_loss': [],
        'train_acc': [],
        'val_acc': []
    }

    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0
        correct_train = 0
        total_train = 0

        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        # Validation
        model.eval()
        val_loss = 0
        correct_val = 0
        total_val = 0

        with torch.no_grad():
            for features, labels in val_loader:
                features, labels = features.to(device), labels.to(device)
                outputs = model(features)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        # Record metrics
        train_loss = train_loss/len(train_loader)
        val_loss = val_loss/len(val_loader)
        train_acc = 100 * correct_train/total_train
        val_acc = 100 * correct_val/total_val

        results['train_loss'].append(train_loss)
        results['val_loss'].append(val_loss)
        results['train_acc'].append(train_acc)
        results['val_acc'].append(val_acc)

        scheduler.step(val_loss)
        early_stopper(val_loss)

        if early_stopper.early_stop:
            print(f"Early stopping at epoch {epoch}")
            break

    return results

def run_experiments(hidden_sizes=[32, 64, 128],
                   num_layers=[1, 2],
                   epochs=[5, 50, 100, 250, 350],
                   optimizers=['sgd', 'rmsprop', 'adam'],
                   learning_rates=[0.001, 0.01]):

    data = pd.read_csv("/content/sample_data/Iris.csv")
    X = data.iloc[:, 1:5].values
    y = pd.Categorical(data.Species).codes

    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    train_dataset = IrisDataset(X_train, y_train)
    test_dataset = IrisDataset(X_test, y_test)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32)

    results = []

    for hidden_size in hidden_sizes:
        for n_layers in num_layers:
            for epoch in epochs:
                for opt in optimizers:
                    for lr in learning_rates:
                        model = BiRNN(4, hidden_size, 3, num_layers=n_layers)
                        criterion = nn.CrossEntropyLoss()

                        if opt == 'sgd':
                            optimizer = torch.optim.SGD(model.parameters(), lr=lr)
                        elif opt == 'rmsprop':
                            optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)
                        else:
                            optimizer = torch.optim.Adam(model.parameters(), lr=lr)

                        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)
                        early_stopper = EarlyStopper(patience=10)

                        training_results = train_model(model, train_loader, test_loader,
                                                     criterion, optimizer, scheduler,
                                                     epoch, early_stopper)

                        # Final evaluation
                        model.eval()
                        correct = 0
                        total = 0
                        with torch.no_grad():
                            for features, labels in test_loader:
                                outputs = model(features)
                                _, predicted = torch.max(outputs.data, 1)
                                total += labels.size(0)
                                correct += (predicted == labels).sum().item()

                        accuracy = 100 * correct / total
                        final_train_loss = training_results['train_loss'][-1]
                        final_val_loss = training_results['val_loss'][-1]

                        results.append({
                            'hidden_size': hidden_size,
                            'num_layers': n_layers,
                            'epochs': epoch,
                            'optimizer': opt,
                            'learning_rate': lr,
                            'accuracy': accuracy,
                            'final_train_loss': final_train_loss,
                            'final_val_loss': final_val_loss
                        })

    return pd.DataFrame(results)

# Run experiments
results = run_experiments()

# Analysis
print("\nAverage Accuracy by Hidden Size:")
print(results.groupby('hidden_size')['accuracy'].mean())

print("\nAverage Accuracy by Number of Layers:")
print(results.groupby('num_layers')['accuracy'].mean())

print("\nAverage Accuracy by Optimizer:")
print(results.groupby('optimizer')['accuracy'].mean())

print("\nAverage Accuracy by Learning Rate:")
print(results.groupby('learning_rate')['accuracy'].mean())

print("\nBest Configuration:")
best_model = results.loc[results['accuracy'].idxmax()]
print(best_model)

Early stopping at epoch 33
Early stopping at epoch 38
Early stopping at epoch 22
Early stopping at epoch 53
Early stopping at epoch 155
Early stopping at epoch 29
Early stopping at epoch 32
Early stopping at epoch 117
Early stopping at epoch 28
Early stopping at epoch 203
Early stopping at epoch 58
Early stopping at epoch 18
Early stopping at epoch 20
Early stopping at epoch 74
Early stopping at epoch 16
Early stopping at epoch 95
Early stopping at epoch 19
Early stopping at epoch 37
Early stopping at epoch 27
Early stopping at epoch 115
Early stopping at epoch 20
Early stopping at epoch 76
Early stopping at epoch 25
Early stopping at epoch 107
Early stopping at epoch 18
Early stopping at epoch 25
Early stopping at epoch 37
Early stopping at epoch 33
Early stopping at epoch 28
Early stopping at epoch 89
Early stopping at epoch 27
Early stopping at epoch 164
Early stopping at epoch 30
Early stopping at epoch 122
Early stopping at epoch 22
Early stopping at epoch 133
Early stopping at ep

### Analisis Hyperparameter


#### **1. Hidden Size**
- **Deskripsi**: Jumlah unit neuron di setiap lapisan tersembunyi (hidden layer).
- **Eksperimen**: Hidden sizes `[32, 64, 128]`.
- **Hasil Analisis**:
  - **Hidden size** yang lebih besar cenderung menghasilkan model dengan kapasitas representasi yang lebih tinggi, namun dapat meningkatkan risiko overfitting pada dataset kecil seperti Iris.
  - Akan terlihat tren bahwa **hidden size** 64 atau 128 mungkin unggul dalam akurasi, tetapi dengan trade-off waktu pelatihan yang lebih lama.

---

#### **2. Number of Layers**
- **Deskripsi**: Jumlah lapisan RNN (stacked RNN).
- **Eksperimen**: `num_layers = [1, 2]`.
- **Hasil Analisis**:
  - Menambahkan lebih banyak lapisan dapat meningkatkan kemampuan representasi jaringan, tetapi dapat menyebabkan eksploding/vanishing gradients jika jumlah lapisan terlalu besar.
  - Pada dataset kecil, jumlah lapisan 1 biasanya sudah cukup, dan lapisan tambahan dapat menghasilkan performa serupa atau sedikit lebih buruk karena overfitting.

---

#### **3. Epochs**
- **Deskripsi**: Jumlah iterasi pelatihan penuh terhadap dataset.
- **Eksperimen**: `epochs = [5, 50, 100, 250, 350]`.
- **Hasil Analisis**:
  - Epoch terlalu rendah (misalnya 5) cenderung menghasilkan underfitting, sementara epoch tinggi (350) berisiko overfitting.
  - Dengan mekanisme early stopping, eksperimen ini secara otomatis menghentikan pelatihan jika validasi loss tidak membaik setelah beberapa iterasi, mengurangi risiko pemborosan waktu pelatihan pada epoch tinggi.

---

#### **4. Optimizers**
- **Deskripsi**: Algoritma optimisasi untuk pembaruan bobot.
- **Eksperimen**: `optimizers = ['sgd', 'rmsprop', 'adam']`.
- **Hasil Analisis**:
  - **SGD**: Cenderung lebih lambat karena menggunakan gradien batch. Namun, dengan learning rate yang tepat, dapat mencapai performa baik.
  - **RMSprop**: Lebih cocok untuk masalah yang melibatkan data sekuensial seperti RNN, karena mengadaptasi learning rate.
  - **Adam**: Biasanya memberikan hasil terbaik karena menggabungkan keunggulan momentum (SGD) dan adaptif learning rate (RMSprop).

---

#### **5. Learning Rate**
- **Deskripsi**: Kecepatan pembaruan bobot selama pelatihan.
- **Eksperimen**: `learning_rates = [0.001, 0.01]`.
- **Hasil Analisis**:
  - Learning rate tinggi (0.01) dapat mempercepat pelatihan tetapi berisiko melewati titik optimal.
  - Learning rate rendah (0.001) lebih stabil tetapi memerlukan waktu pelatihan lebih lama.

---

#### **6. Dropout**
- **Deskripsi**: Teknik regularisasi untuk mencegah overfitting.
- **Pengaturan**: Default dropout = `0.1`.
- **Hasil Analisis**:
  - Dropout 0.1 mungkin cukup untuk dataset kecil. Dropout lebih tinggi (misal, 0.5) tidak dicoba, tetapi bisa lebih berguna untuk dataset yang lebih besar atau lebih kompleks.

---

#### **7. Scheduler**
- **Deskripsi**: Strategi penurunan learning rate berdasarkan metrik validasi.
- **Pengaturan**: `ReduceLROnPlateau` dengan `factor=0.1` dan `patience=5`.
- **Hasil Analisis**:
  - Scheduler ini membantu mencegah pemborosan waktu pada learning rate yang tidak efektif dengan secara dinamis menurunkannya.

---

#### **Kesimpulan Eksperimen**
- **Rata-rata akurasi** dapat digunakan untuk memahami tren optimal dari hyperparameter tertentu.
- **Kombinasi terbaik** dapat diidentifikasi dengan **akurasi tertinggi**. Dari hasil kode:
  - **Hidden Size**: 64 atau 128.
  - **Number of Layers**: 1.
  - **Optimizer**: Adam.
  - **Learning Rate**: 0.001.
  - **Epochs**: Sesuai early stopping (biasanya di bawah 100 untuk dataset kecil).

---

#### **Output yang Perlu Dicatat**
1. **Rata-rata akurasi berdasarkan masing-masing hyperparameter.**
2. **Kombinasi hyperparameter terbaik dengan akurasi tertinggi.**
3. **Final training loss dan validation loss dari konfigurasi terbaik.**

